# 2.4 Data Acquision BDTNS

Goal of this notebook is to transform [BDTNS](http://bdtns.filol.csic.es/) data into structured format that clearly distinguishes between text and non-text (such as line numbers) and that, for the text part, follows as much as possible the standards of the Oracc Global Sign List([OGSL](http://oracc.org/ogsl)). Adherence to the [OGSL](http://oracc.org/ogsl) standard makes it possible to transform a line of text into a sequence of sign names or Unicode codepoints. This can be used to track transliteration inconsistencies, for instance in the rendering of names, as in U₂-da-mi-ša-ru-um vs. U₂-ta₂-mi-ša-ru-um, to build a search engine that finds a sign sequence regardless of the actual transliteration of the individual signs, or to compare editions of the same text in [BDTNS](http://bdtns.filol.csic.es/), [CDLI](http://cdli.ucla.edu), and [ePSD2](http://oracc.org/epsd2/admin/u3adm).

On the [BDTNS](http://bdtns.filol.csic.es/) website, search results can be downloaded with "Export" button in the left pane. Searching for the empty string will select all texts currently available. The export creates two files: one with transliterations and one with meta-data. The transliteration file is discussed here most extensively. The file name is "query_text_" followed by a date and an additional number. Move the file to the `data` directory of this chapter and make sure that the variable `file` corresponds to your file name and file location.

In [1]:
import pandas as pd
import regex
from tqdm import tqdm

# Read the File as a List
The [BDTNS](http://bdtns.filol.csic.es/) export file is read as a list (each line is one element in the list) with the `splitlines()` function. [BDTNS](http://bdtns.filol.csic.es/) uses so-called "vertical tabs" represented by `^K`, `\v`, or `\x0b`, depending on your editor. The vertical tab is used as a new line within a single text. The Python `readlines()` function (which reads a file line-by-line, producing a list) keeps the vertical tabs and will result in a format where each document is represented as a single line. Instead, therefore, we read the entire text with the `read()` function and then split the document into lines with `splitlines()`. The function `splitlines()` takes the vertical tab (as well as the regular newline character) as a line separator; this will result in a line-by-line representation of the document in the form of a list (called `bdtns`).

In [2]:
file = 'query_text_19_05_15-020951.txt'
with open('data/' + file, mode = 'r', encoding = 'utf8') as f:
    bdtns = f.read().splitlines()

# Remove empty lines
Empty lines (or lines filled with spaces or tabs only) will cause trouble downstream and are removed.

In [3]:
bdtns = [line for line in bdtns if len(line.strip()) > 0]

# Make DataFrame
Line numbers are separated from transliteration text by 5 spaces. Editorial remarks are introduced by the hash-mark (#). Finally, sign explications (of x-values, rare signs, and, occasionally, of readings with exclamation mark) are marked by "(=". The five spaces are replaced by '#' and '(=' is replaced by '#(=' so that we can split on the hash mark. The `replace()` function is done only once and `split()` is done twice, so that we end up with three columns, representing line label, text, and comments. The column 'comments' will include both editorial comments and sign explications of x-values.

The [BDTNS](http://bdtns.filol.csic.es) number is added in the first column. Other meta-data (such as publication and [CDLI](http://cdli.ucla.edu) P-number) are omitted, because they are better derived from the meta-data file (with the [BDTNS](http://bdtns.filol.csic.es) number as key).

In [4]:
l = []
bdtns_no = ""
for line in bdtns: 
    if line[:6].isdigit(): 
        bdtns_no = line[:6]
        continue
    else: 
        li = line.strip()
        li = li.replace("(=", "#(=", 1).replace('     ', '#', 1)
        li = li.split('#', 2)
        li = [bdtns_no] + li
        l.append(li)

In [5]:
columns = ["bdtns_no", "line_label", "text", "comments"]
df = pd.DataFrame(l, columns=columns).fillna("")

# Make OGSL compliant
[OGSL](http://oracc.org/ogsl) is the ORACC Global Sign List, which lists for each sign its possible readings. [OGSL](http://oracc.org/ogsl) compliance opens the possibility to search or compare by sign *name* rather than sign value. For instance, one may search for the sequence "aga₃ kug-sig₁₇" (golden tiara) and find a line reading "gin₂ ku₃-GI". [ORACC](http://oracc.org) projects are [OGSL](http://oracc.org) compliant by definition (a project will not build if it does not follow [OGSL](http://oracc.org/ogsl) rules) and [CDLI](http://cdli.ucla.edu) data can easily be transformed to [OGSL](http://oracc.org/ogsl) compliance. Transformation of [BDTNS](http://bdtns.filol.csic.es) data to [OGSL](http://oracc.org/ogsl) compliance, therefore, also makes it possible to compare the editions of the same text in the three projects.

The main steps towards [OGSL](http://oracc.org/ogsl) compliance are: 

- replace regular number by index numbers in sign values
- add sign explication to x-values


# Dealing with x-values
in [OGSL](http://oracc.org/ogsl) a sign reading that has not (yet) received a commonly accepted index receives the ₓ index, followed by the sign name, as in ziₓ(SIG₇). In [BDTNS](http://bdtns.filol.csic.es) the subscripted ₓ is represented by a capital X and the sign name is given at the end of the line, as in

> o. 2     gi ziX-a 12 sar-⌈ta⌉ (=SIG7)

Because of the way the DatafRame has been produced, the sign explications are now found in the column `comments`. The most straightforward solution is to replace every capital X with an ₓ plus what is found in the comments column (minus the equal sign).

This will result in: 
> o. 2     gi ziₓ(SIG₇)-a 12 sar-⌈ta⌉

In practice, however, there are quite a few exceptions where the pattern presented above is not followed. Examples include

> 18 gin2 nagga mu-kuX gibil (=AN.NA) (=DU)

Here (=AN.NA) explains the infrequent sign "nagga". However, since "nagga" is not followed by X the regular expression will not recognize that and this will result in

> 18 gin₂ nagga mu-kuₓ(AN.NA) gibil (=DU)

Another exception is reduplicated "gurₓ-gurₓ" which is represented thus: 
> 6.0.0 še ur5-ra še gurX-gurX-ta su-ga (=ŠE.KIN.ŠE.KIN)

resulting in: 
> 6.0.0 še ur₅-ra še gurₓ(ŠE.KIN.ŠE.KIN)-gurₓ-ta su-ga

Finally, a few x-values are usually not resolved this way, for instance: 
> 1 sila4 Ur-nigarX{gar} \[...\]

with no explication of the nigarX - presumably because it is unambiguous.

For these reasons we will approach the x-values in two different ways

- x-values that resolve unambiguously are resolved with a simple search and replace, replacing, e.g. ziₓ with ziₓ(SIG₇), without paying attention to the [BDTNS](http://bdtns.filol.csic.es) sign explication (=SIG7). A special case in this category is mu-kuX ("delivery"), which is very frequent and should be resolved as mu-kuₓ(DU). However, kuX by itself may also be resolved as kuX(LIL) or kuₓ(KWU147) (for the verb "to enter").
- x-values that do not resolve unambiguously (muruₓ, ušurₓ, ummuₓ, and several others) are resolved by moving the [BDTNS](http://bdtns.filol.csic.es) sign explanation (in the `comments` column) after the X sign between brackets, as explained above.

Both these steps are included in a single function (`ogsl_v()`) that is applied to every row of the DataFrame. In addition, this function will replace index numbers (such as the 7 in SIG7) with Unicode index numbers (SIG₇).

# Step 1: Unambiguous x-values

Some x-values are always resolved in the same way. Thus, ziX is always ziₓ(SIG₇), hirinX is always hirinₓ(KWU318), and gurX is always gurₓ(|ŠE.KIN|). In some cases, x-values have been assigned an index number in [OGSL](http://oracc.org/ogsl). In those cases (nigarₓ = nigar; nemurₓ(PIRIG.TUR) = nemur₂; nagₓ(GAZ) = nag₃; and pešₓ(ŠU.PEŠ5) = peš₁₄) the appropriate index number is added and the sign explication is discarded.

A dictionary of such unambiguous x-values (`xvalues`) has its key the x-value as represented in [BDTNS](http://bdtns.filol.csic.es) ('ziX') and as its value the index ₓ plus the appropriate sign explication ('ₓ(SIG₇)') in [OGSL](http://oracc.org/ogsl) format.

For various reasons the substitution cannot be done with the general `replace()` function. First, instead of 'ziₓ' we may encounter \[z\]iₓ or some other usage of flags that indicate certainty or breakage. Second, 'subX' should not match 'munsubX'. Third, not every X indicates an x-value - X is also used (less frequently) for illegible signs, as in KA×X, or simply X.

As a first step, a line is split into words with the `split()` function (by default this will split on spaces). The for loop then iterates through the `xvalues` dictionary. First, it tests whether the sought-for x-value is present in the line - if not, we go to the next x-value in the dictionary. Because of possible flags, the test is not done on the actual line, but on a "translated" version of the line. The function `translate()` takes a table and translates characters to other characters. In the present case, each character is replaced by `None`- which means they are removed. Note that the flags are not removed from the actual transliteration - they are only removed in the `if` statement to test for the presence of the target string. The search for the target word is done with a regular expression that surrounds the word with `\\b` and `\\b`. The `\\b` represents the "word boundary". Note that in regular expressions word boundaries include the hyphen, and various brackets, so that '\\bgurₓ\\b'matches with 'in-gurX'.

If the target word is present in the line, the same test is executed on each (translated) word of the line and once the word is found the uppercase X is replaced by the corresponding value in the dictionary `xvalues`.

After the script has iterated through all the items in the `xvalues` dictionary the full line is put together again (with the `join()` function) and the second part of the function will look at remaining x-values.

# Step 2: Remaining x-values
For the remaining x-values (many of them ambiguous) we will copy the [BDTNS](http://bdtns.filol.csic.es) sign explication, found in the `comments` column, to the x-value. For instance, **ummu₃** is |A.EDIN.LAL|, but the sign complex has many variants, all rendered **ummuX**: EDIN.A.SU, A.EDIN, A.EDIN.A.LAL, EDIN, etc. The code will result in ummuₓ(|A.EDIN.SU|), ummuₓ(|A.EDIN|), ummuₓ(|A.EDIN.LAL|), ummuₓ(EDIN), etc. Compound signs are put between pipes (|A.EDIN.SU|), accoring to [OGSL](http://oracc.org/ogsl) conventions.

In this step the code will naively replace the capital X by the index ₓ, followed by the first word in the `comments` column. This will result in errors if there are more such x-values in a single line - but because we have already dealt with many such values in the preceding, that risk is not very high. The code will test that the capital X does in fact follow a sign reading (as in ziX), and is not an illegible sign (as in KA×X, or simply X). This is done with a [regular expression](https://www.regular-expressions.info/) using a so-called "positive lookbehind" (?<=), to see if the preceding character is a letter. The sequence \\p{L} indicates any letter in any Unicode alphabet - but excludes numbers and operators, etc. This convention is not available in the `re` library (the most commonly used library for regular expressions), but it can be used in `regex` library, which replaces `re` in cases where more expansive unicode support is necessary.

# Step 3: Index Numbers
In a third step all sign reading index numbers (as in 'du11') are replaced by Unicode index numbers ('du₁₁'). Regular numbers that express quantities should not be affected. This done, again, with a look-behind regular expression that identifies the character immediately before the number as a (Unicode) letter. If such a match is found, the number is replaced by the index number. The process needs to be done twice, in order to take care of double digit index numbers. In the second round, the look-behind regular expression looks for index numbers in the range [₀-₉].

# Errors
Inevitably, each of the steps in dealing with x-values may introduce its own errors. It is likely, moreover, that there are more x-values not treated here, or that there will be more x-values in a future version of the [BDTNS](http://bdtns.filol.csic.es) data. The lits of x-values below can be adapted to deal with those situations. 

In [6]:
xvalues = {'nagX' : '₃', 'nigarX' : '', 'nemurX' : '₂', 'pešX' : '₁₄',
        'ziX' : 'ₓ(SIG₇)' ,  'urubX' : 'ₓ(|URU×KAR₂|)',
        'alX' : 'ₓ(TUR₃)' , 'sagX' : 'ₓ(|ŠE.KIN|)', 'zahX' : 'ₓ(ŠEŠ)', 
        'šaganX' : 'ₓ(|AMA.GAN|)', 'sullimX' : 'ₓ(EN)',
        'gurX' : 'ₓ(|ŠE.KIN|)', 'niginX' : 'ₓ(|LAL₂×SAR|)',
        'gišbunX' : 'ₓ(|KI.BI|)',  
        'girX' : 'ₓ(GI)', 'gigirX' : 'ₓ(|LAGAB×MU|)',
        'tubaX' : 'ₓ(TUG₂)', 'giparX' : 'ₓ(KISAL)',
        'zabalamX' : 'ₓ(|MUŠ₃.TE.UNUG|)', 'duruX' : 'ₓ(U₃)',
        'dagX' : 'ₓ(KWU844)', 'hirinX' : 'ₓ(KWU318)', 
        'bulugX' : 'ₓ(|ŠIM×KUŠU₂|)', 'subX' : 'ₓ(|DU.DU|)',
        'munsubX' : 'ₓ(|PA.USAN|)', 'kurunX' : 'ₓ(|DIN.KAŠ|)', 'mu-kuX' : 'ₓ(DU)', 
        'zahdaX' : 'ₓ(|ŠAH₂.NE.TUR|)', 'šuX' : 'ₓ(TAG)', 'ulušinX' : 'ₓ(|BI.AŠ|)', 
        'durunX' : 'ₓ(|KU.KU)'}
# TODO: make sure that all sign explications follow OGSL standard and use standard sign names (KAR₂ should be GAN₂@g?)

In [7]:
flags = "][!?<>⸢⸣⌈⌉*"
table = str.maketrans(dict.fromkeys(flags))

In [8]:
nos = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'X']
indexes = ['₀', '₁', '₂', '₃', '₄', '₅', '₆', '₇', '₈', '₉', 'ₓ']
ind_d = dict(zip(nos, indexes))

In [9]:
def ogsl_v(row):
    # 1. deal with unambiguous x-values, listed in the dictionary xvalues.
    m = row['text'].split()  # split line into words
    for value in xvalues: 
        if not regex.findall('\\b' + value.lower() +'\\b', row['text'].translate(table).lower()): 
            continue
        else: 
            for i, w in enumerate(m):
                if not regex.findall('\\b' + value.lower() +'\\b', w.translate(table).lower()): 
                    continue
                else: 
                    m[i] = w.replace("X", xvalues[value])
    row['text'] = ' '.join(m)
    
    # 2. deal with remaining x-values
    if row["comments"][:2] == "(=": 
        sign_explic = row["comments"].split()[0][2:-1]  # remove (= and ) from (=SIG7)
        if '.' in sign_explic or '×' in sign_explic: 
            sign_explic = 'ₓ(|' + sign_explic + '|)'  # add pipes if necessary
        else: 
            sign_explic = 'ₓ(' + sign_explic + ')'
        ogsl_valid = regex.sub('(?<=[\p{L}])X', sign_explic, row['text'])
    else:
        ogsl_valid = row["text"]
    
    # 3 deal with index numbers
    for no in ind_d: 
        ogsl_valid = regex.sub('(?<=[\p{L}])' + no, ind_d[no], ogsl_valid)
    for no in ind_d: 
        ogsl_valid = regex.sub('(?<=[₀-₉])' + no, ind_d[no], ogsl_valid)
    return ogsl_valid

# Apply the Function
The `ogsl_v` function is now applied to each row in the DataFrame. The DataFrame currently has more than 1.1 million rows (lines) and the function is quite involved, so this will take some time. For that reason a progress bar has been added. The progress bar is part of the `tqdm` library. It is initialized with the line
```python
tqdm.pandas()
```
Instead of the regular `apply()` function from the `pandas` library we may now use `progress_apply()` to do the same thing as `apply()`, but with a progress bar.

In [10]:
tqdm.pandas() 
df["text"] = df.progress_apply(ogsl_v, axis = 1) 

100%|███████████████████████████████| 1156363/1156363 [39:36<00:00, 486.68it/s]


# Check for Remaining x-values

In [13]:
df[df["text"].str.contains("X")]

,bdtns_no,line_label,text,comments
1203,038652,r. 29,[sa₂]-⌈du₁₁⌉ {d}En-ki u₃ {d}Uš-KA×X?-limmu₂,
1701,038660,r. 22',KA×X? la₂?-la₂? ⌈ša₃⌉ bala-a,
4200,038754,r. 8,giri₃ bad₃-HI×X dumu LAGAB-ba-⌈x⌉-gu₂,
4380,038763,o. 5,[...]-{d}Nanna mu / ⌈lugal⌉ PU₃ KA×X ⌈x⌉ [...],
27306,002492,r. 13,ša₃ e₂ KA×X-⌈x⌉-[...],
29168,023466,o. 4,GIŠ.X.KI//NA-i₃-sa₆ sukkal gaba-aš,
56326,028750,o. 5,6.0.0 gur šuku Puzur₄-{d}GIŠ×X ra₂-gaba,
56690,029061,o. 5,{dug}KISIM₅×X x,
67424,029089,o. 6,Ba-za lu₂-KA×X,
75213,034930,o.iii 7,a-ša₃ A.LAGAB×X.TUR,


In [14]:
df[df["text"].str.contains('ₓ')]

,bdtns_no,line_label,text,comments
88,038574,o. 2,gi ziₓ(SIG₇)-a 12 sar-⌈ta⌉,(=SIG7)
99,038575,o. 3,4 guruš gi ziₓ(SIG₇)-a 12 sar-ta,(=SIG7)
118,038577,o. 6,la₂-i₃ su-ga Lugal-ušurₓ(|LAL₂.TUG₂|) sipa,(=LAL2.TUG2)
233,038586,o. 4,1 Lu₂-Urubₓ(|URU×KAR₂|){ki},(=URU×KAR2)
452,038605,r. 5,Umma{ki}-a mu-kuₓ(DU),(=DU)
535,038611,r. 4,ša₃ Tum-alₓ(TUR₃){ki},(=TUR3)
580,038614,r. 14,mu-kuₓ(DU),(=DU)
601,038616,o. 7,⌈24⌉ ⌈sar⌉ tug₂-sagₓ(|ŠE.KIN|) ⌈x⌉,(=ŠE.KIN)
897,038635,o. 3,mu-kuₓ(DU) {d}Inanna Zabalam₄{ki},(=DU) (=MUŠ3.ZA.UNUG)
1269,038654,r. 19,mu-kuₓ(DU) libir-⌈a⌉,(=DU)
